In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# 定义文件夹路径和文件扩展名
folder_path = r"." #"D:\Jupiter_eyemovement_data\Pic_TOJ\formal\SESSION1\ASC"
file_extension = ".asc"

# 获取所有ASC文件的路径列表
asc_file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path) if file_name.endswith(file_extension)]
# asc_file_paths

# # 对每个ASC文件执行批处理操作
# for asc_file_path in asc_file_paths:
#     # 执行你要进行的批处理操作
#     # 以下示例操作是将文件复制到一个新文件夹中
#     new_folder_path = os.path.join(folder_path, "processed")
#     os.makedirs(new_folder_path, exist_ok=True)
#     file_name = os.path.basename(asc_file_path)
#     new_file_path = os.path.join(new_folder_path, file_name)
#     shutil.copy(asc_file_path, new_file_path)


In [3]:
def is_number(line):
    try: 
        int(line[0])
    except ValueError:
        return False
    return True

def is_msg(line):
    if line[0:3] == 'MSG':
        return True
    return False 

def is_newline(line):
    if line == '' or line == '\n':
        return True
    return False

def is_fix(line):
    if "SFIX" in line or "EFIX" in line:
        return True
    return False

def is_sacc(line):
    if "SSACC" in line or "ESACC" in line:
        return True
    return False

def is_blink(line):
    if "SBLINK" in line or "EBLINK" in line:
        return True
    return False    

In [4]:
file_to_parse = "./J030701.asc"
print("start parsing file %s"%file_to_parse)
file = open(file_to_parse, 'r')
lines = file.read().splitlines(True)
header_split = 0
for i in range(0, len(lines) - 2):
    if is_msg(lines[i]) & is_newline(lines[i+1]) & is_number(lines[i+2]):
        header_split = i + 2
        break

original_removed_header = np.array(lines[header_split:])
print('finish remove header')


start parsing file ./J030701.asc
finish remove header


In [5]:
original_size = np.size(original_removed_header)
print(original_size)

749351


In [6]:
msg = []
fix = []
sacc = []
blink = []
miss = []
removed_header = original_removed_header

In [7]:
for i in reversed(range(np.size(removed_header))):
    if is_number(removed_header[i]):
        continue
    else:
        if is_msg(removed_header[i]):
            msg.append(removed_header[i])
        elif is_blink(removed_header[i]):
            blink.append(removed_header[i])
        elif is_fix(removed_header[i]):
            fix.append(removed_header[i])
        elif is_sacc(removed_header[i]):
            sacc.append(removed_header[i])
        else:
            miss.append(removed_header[i])
        removed_header = np.delete(removed_header, i, 0)
print(len(msg) + len(fix) + len(blink) + len(sacc) + len(miss) + np.size(removed_header) == original_size)

True


In [114]:
nparr = np.asarray(removed_header)
out_arr = np.char.split(nparr)

In [123]:
arr = np.asarray([np.array(line) for line in out_arr])

6

In [126]:
remove_dot = arr[:, :-1]
pd.DataFrame(remove_dot).to_csv('sample.csv', index_label = "Index", header  = ['Time','x','y','pupil size','CR'])

In [157]:
print(msg)
for i in range(len(msg)):
    msg[i] = msg[i].replace('\t', ' ')
print(msg)
split_msg = np.char.split(msg)
for i in range(len(split_msg)):
    if len(split_msg[i])>3:
        for str in split_msg[i][3:]:
            split_msg[i][2] += " "
            split_msg[i][2] += str
            split_msg[i].remove(str)
print(split_msg)
np_msg = np.asarray([np.array(line) for line in split_msg])
np_msg

['MSG 333976184 jitter_iti_end_3\n', 'MSG 333974623 jitter_iti_start_3\n', 'MSG 333972630 reward_end_3\n', 'MSG 333972630 !V CLEAR 0 0 0\n', 'MSG 333970633 !V IMGLOAD CENTER F:\\Pic_TOJ\\pic_pool_step_3_formal\\682_3.png 1280 972 576 320\n', 'MSG 333970633 reward_start_3\n', 'MSG 333970614 forcedchoice_end_3\n', 'MSG 333970614 !V CLEAR 0 0 0\n', 'MSG 333968914 !V DRAWBOX 0 255 0 1845 537 1995 687\n', 'MSG 333968914 !V DRAWBOX 0 255 0 565 537 715 687\n', 'MSG 333968914 !V DRAWBOX 0 255 0 1205 1257 1355 1407\n', 'MSG 333968914 !V IAREA RECTANGLE 3 1632 92 2208 412 right_roi\n', 'MSG 333968914 !V IAREA RECTANGLE 2 352 92 928 412 left_roi\n', 'MSG 333968914 !V IAREA RECTANGLE 1 992 812 1568 1132 bottom_roi\n', 'MSG 333968914 !V IMGLOAD CENTER F:\\Pic_TOJ\\pic_pool_step_3_formal\\682_1.png 1920 251 576 320\n', 'MSG 333968914 !V IMGLOAD CENTER F:\\Pic_TOJ\\pic_pool_step_3_formal\\682_2.png 640 251 576 320\n', 'MSG 333968913 !V IMGLOAD CENTER F:\\Pic_TOJ\\pic_pool_step_3_formal\\682_3.png 128

array([['MSG', '333976184', 'jitter_iti_end_3'],
       ['MSG', '333974623', 'jitter_iti_start_3'],
       ['MSG', '333972630', 'reward_end_3'],
       ...,
       ['MSG', '333604981', 'selfpaced_start_m_end'],
       ['MSG', '333603697', 'selfpaced_start_m_start'],
       ['MSG', '333603697', 'EYE_USED 1 RIGHT']], dtype='<U79')

In [158]:
pd.DataFrame(np_msg).to_csv('msg.csv', index_label = "Index", header  = ['Type','Time','Desceiption'])

In [27]:
print(fix)
for i in range(len(fix)):
    fix[i] = fix[i].replace('\t', ' ')
split_fix = np.char.split(fix)
print(np.shape(split_fix)[0])
for i in reversed(range(np.shape(split_fix)[0])):
    if split_fix[i][0] == "SFIX":
        split_fix = np.delete(split_fix, i, 0)
print(split_fix)

['EFIX R   333976098\t333976193\t95\t 1228.1\t  706.9\t   5106\n', 'SFIX R   333976098\n', 'EFIX R   333975752\t333976046\t294\t  517.3\t  471.5\t   4888\n', 'SFIX R   333975752\n', 'EFIX R   333975447\t333975685\t238\t 1276.7\t  712.5\t   5342\n', 'SFIX R   333975447\n', 'EFIX R   333975234\t333975426\t192\t 1312.2\t  700.5\t   5243\n', 'SFIX R   333975234\n', 'EFIX R   333974940\t333975184\t244\t  674.2\t  438.0\t   5087\n', 'SFIX R   333974940\n', 'EFIX R   333974646\t333974878\t232\t 1273.8\t  714.7\t   5255\n', 'SFIX R   333974646\n', 'EFIX R   333974456\t333974626\t170\t 1305.1\t  748.7\t   5225\n', 'SFIX R   333974456\n', 'EFIX R   333974233\t333974404\t171\t 2011.7\t  247.4\t   4836\n', 'SFIX R   333974233\n', 'EFIX R   333973409\t333974165\t756\t 2332.5\t 1102.5\t   4713\n', 'SFIX R   333973409\n', 'EFIX R   333972822\t333973144\t322\t 1256.9\t  725.9\t   4529\n', 'SFIX R   333972822\n', 'EFIX R   333972141\t333972785\t644\t 1055.0\t  919.6\t   4360\n', 'SFIX R   333972141\n',

In [163]:
import csv
with open('FIX.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerows(split_fix)

In [28]:
np_efix = np.asarray([np.array(line) for line in split_fix])
reverse_efix = np_efix[::-1]
reverse_efix
pd.DataFrame(reverse_efix).to_csv('efix.csv', index_label = "Index", header  = ['Type','Eye','Start_time','End_time','Duration','Avg_x', 'Avg_y', 'Angle'])

In [9]:
print(miss)
for i in range(len(miss)):
    miss[i] = miss[i].replace('\t', ' ')
split_miss = np.char.split(miss)


['END\t333976195 \tSAMPLES\tEVENTS\tRES\t  38.00\t  38.25\n', 'INPUT\t333976194\t127\n', '\n', 'INPUT\t333907981\t127\n', 'SAMPLES\tGAZE\tRIGHT\tRATE\t2000.00\tTRACKING\tCR\tFILTER\t2\tINPUT\n', 'EVENTS\tGAZE\tRIGHT\tRATE\t2000.00\tTRACKING\tCR\tFILTER\t2\n', 'PUPIL\tAREA\n', 'VPRESCALER\t1\n', 'PRESCALER\t1\n', 'START\t333907981 \tRIGHT\tSAMPLES\tEVENTS\n', 'END\t333907943 \tSAMPLES\tEVENTS\tRES\t  38.57\t  38.44\n', 'INPUT\t333907942\t127\n', '\n', 'INPUT\t333834869\t127\n', 'SAMPLES\tGAZE\tRIGHT\tRATE\t2000.00\tTRACKING\tCR\tFILTER\t2\tINPUT\n', 'EVENTS\tGAZE\tRIGHT\tRATE\t2000.00\tTRACKING\tCR\tFILTER\t2\n', 'PUPIL\tAREA\n', 'VPRESCALER\t1\n', 'PRESCALER\t1\n', 'START\t333834869 \tRIGHT\tSAMPLES\tEVENTS\n', 'END\t333834830 \tSAMPLES\tEVENTS\tRES\t  40.37\t  39.59\n', 'INPUT\t333834829\t127\n', '\n', 'INPUT\t333753500\t127\n', 'SAMPLES\tGAZE\tRIGHT\tRATE\t2000.00\tTRACKING\tCR\tFILTER\t2\tINPUT\n', 'EVENTS\tGAZE\tRIGHT\tRATE\t2000.00\tTRACKING\tCR\tFILTER\t2\n', 'PUPIL\tAREA\n', 'VP

array([list(['END', '333976195', 'SAMPLES', 'EVENTS', 'RES', '38.00', '38.25']),
       list(['INPUT', '333976194', '127']), list([]),
       list(['INPUT', '333907981', '127']),
       list(['SAMPLES', 'GAZE', 'RIGHT', 'RATE', '2000.00', 'TRACKING', 'CR', 'FILTER', '2', 'INPUT']),
       list(['EVENTS', 'GAZE', 'RIGHT', 'RATE', '2000.00', 'TRACKING', 'CR', 'FILTER', '2']),
       list(['PUPIL', 'AREA']), list(['VPRESCALER', '1']),
       list(['PRESCALER', '1']),
       list(['START', '333907981', 'RIGHT', 'SAMPLES', 'EVENTS']),
       list(['END', '333907943', 'SAMPLES', 'EVENTS', 'RES', '38.57', '38.44']),
       list(['INPUT', '333907942', '127']), list([]),
       list(['INPUT', '333834869', '127']),
       list(['SAMPLES', 'GAZE', 'RIGHT', 'RATE', '2000.00', 'TRACKING', 'CR', 'FILTER', '2', 'INPUT']),
       list(['EVENTS', 'GAZE', 'RIGHT', 'RATE', '2000.00', 'TRACKING', 'CR', 'FILTER', '2']),
       list(['PUPIL', 'AREA']), list(['VPRESCALER', '1']),
       list(['PRESCALER', '1

In [10]:
import csv
with open('miss.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerows(split_miss)

In [14]:
print(sacc)
for i in range(len(sacc)):
    sacc[i] = sacc[i].replace('\t', ' ')
split_sacc = np.char.split(sacc)
print(split_sacc)

['ESACC R  333976047\t333976097\t50\t  514.2\t  477.7\t 1227.1\t  716.3\t  20.18\t   1065\n', 'SSACC R  333976047\n', 'ESACC R  333975686\t333975751\t65\t 1277.1\t  715.0\t  509.6\t  460.2\t  21.68\t    587\n', 'SSACC R  333975686\n', 'ESACC R  333975427\t333975446\t19\t 1313.3\t  691.2\t 1271.9\t  715.5\t   1.38\t    126\n', 'SSACC R  333975427\n', 'ESACC R  333975185\t333975233\t48\t  674.6\t  442.1\t 1314.4\t  716.8\t  19.14\t    818\n', 'SSACC R  333975185\n', 'ESACC R  333974879\t333974939\t60\t 1273.9\t  718.6\t  681.1\t  438.1\t  18.05\t    581\n', 'SSACC R  333974879\n', 'ESACC R  333974627\t333974645\t18\t 1307.4\t  747.0\t 1272.8\t  716.4\t   1.32\t    128\n', 'SSACC R  333974627\n', 'ESACC R  333974405\t333974455\t50\t 1998.9\t  239.2\t 1312.6\t  755.0\t  23.10\t   1011\n', 'SSACC R  333974405\n', 'ESACC R  333974166\t333974232\t66\t 2344.0\t 1096.6\t 2029.9\t  264.8\t  22.05\t    649\n', 'SSACC R  333974166\n', 'ESACC R  333973145\t333973408\t263\t 1258.5\t  740.9\t 2319.2\

In [15]:
import csv
with open('sacc.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerows(split_sacc)

In [17]:
for i in reversed(range(np.shape(split_sacc)[0])):
    if split_sacc[i][0] == "SSACC":
        split_sacc = np.delete(split_sacc, i, 0)
print(split_sacc)

[list(['ESACC', 'R', '333976047', '333976097', '50', '514.2', '477.7', '1227.1', '716.3', '20.18', '1065'])
 list(['ESACC', 'R', '333975686', '333975751', '65', '1277.1', '715.0', '509.6', '460.2', '21.68', '587'])
 list(['ESACC', 'R', '333975427', '333975446', '19', '1313.3', '691.2', '1271.9', '715.5', '1.38', '126'])
 list(['ESACC', 'R', '333975185', '333975233', '48', '674.6', '442.1', '1314.4', '716.8', '19.14', '818'])
 list(['ESACC', 'R', '333974879', '333974939', '60', '1273.9', '718.6', '681.1', '438.1', '18.05', '581'])
 list(['ESACC', 'R', '333974627', '333974645', '18', '1307.4', '747.0', '1272.8', '716.4', '1.32', '128'])
 list(['ESACC', 'R', '333974405', '333974455', '50', '1998.9', '239.2', '1312.6', '755.0', '23.10', '1011'])
 list(['ESACC', 'R', '333974166', '333974232', '66', '2344.0', '1096.6', '2029.9', '264.8', '22.05', '649'])
 list(['ESACC', 'R', '333973145', '333973408', '263', '1258.5', '740.9', '2319.2', '1118.3', '28.34', '641'])
 list(['ESACC', 'R', '3339727

In [25]:
np_esacc = np.asarray([np.array(line) for line in split_sacc])
reverse_esacc = np_esacc[::-1]
reverse_esacc
pd.DataFrame(reverse_esacc).to_csv('esacc.csv', index_label = "Index", header  = ['Type','Eye','Start_time','End_time','Duration','cood_1', 'cood_2', 'cood_3', 'cood_4','cood_5', 'cood_6'])

In [19]:
print(blink)
for i in range(len(blink)):
    blink[i] = blink[i].replace('\t', ' ')
split_blink = np.char.split(blink)
print(split_blink)

['EBLINK R 333973163\t333973347\t184\n', 'SBLINK R 333973162\n', 'EBLINK R 333965674\t333965889\t215\n', 'SBLINK R 333965673\n', 'EBLINK R 333957393\t333957528\t135\n', 'SBLINK R 333957392\n', 'EBLINK R 333955158\t333955300\t142\n', 'SBLINK R 333955157\n', 'EBLINK R 333941429\t333943602\t2173\n', 'SBLINK R 333941429\n', 'EBLINK R 333929157\t333930401\t1244\n', 'SBLINK R 333929156\n', 'EBLINK R 333923174\t333923259\t85\n', 'SBLINK R 333923173\n', 'EBLINK R 333915200\t333915305\t105\n', 'SBLINK R 333915199\n', 'EBLINK R 333899317\t333899428\t111\n', 'SBLINK R 333899316\n', 'EBLINK R 333887112\t333887228\t116\n', 'SBLINK R 333887111\n', 'EBLINK R 333880620\t333880800\t180\n', 'SBLINK R 333880619\n', 'EBLINK R 333867367\t333867501\t134\n', 'SBLINK R 333867366\n', 'EBLINK R 333862691\t333862815\t124\n', 'SBLINK R 333862690\n', 'EBLINK R 333856677\t333856793\t116\n', 'SBLINK R 333856676\n', 'EBLINK R 333851952\t333852055\t103\n', 'SBLINK R 333851951\n', 'EBLINK R 333848729\t333850028\t1299\n

In [20]:
with open('blink.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerows(split_blink)

In [24]:
for i in reversed(range(np.shape(split_blink)[0])):
    if split_blink[i][0] == "SBLINK":
        split_blink = np.delete(split_blink, i, 0)
print(split_blink)
np_blink = np.asarray([np.array(line) for line in split_blink])
reverse_blink = np_blink[::-1]
reverse_blink
pd.DataFrame(reverse_blink).to_csv('eblink.csv', index_label = "Index", header  = ['Type','Eye','Start_time','End_time','Duration'])

# Start matching marker and data

In [45]:
#data
data = np.genfromtxt("data.csv", delimiter=',', dtype="i8,S5,S5,")[1:][:]
data[0]
# for record in data:
#     time = record[1]
#     print(time)

C:\Users\River\AppData\Local\Temp\ipykernel_16724\3447480263.py:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  data = np.genfromtxt("data.csv", delimiter=',', dtype=None)[1:][:]


array([b'0', b'333603557', b'1146.1', b'731.6', b'4141.0', b'127.0'],
      dtype='|S10')

In [59]:
import pandas as pd
headers = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6']
data_f = pd.read_csv('data.csv', sep=',', skiprows = 1, header=None, names=headers, dtype={"col1": int, "col2": float, "col3": str, "col4": str, "col5": float, "col6": float})
data = data_f.to_numpy()
print(data)

[[0 333603557.0 '1146.1' '731.6' 4141.0 127.0]
 [1 333603557.0 '1145.8' '731.5' 4150.0 127.0]
 [2 333603558.0 '1145.8' '730.0' 4160.0 127.0]
 ...
 [744956 333976193.0 '1234.4' '705.1' 5144.0 127.0]
 [744957 333976193.0 '1234.1' '705.1' 5139.0 127.0]
 [744958 333976194.0 '1233.8' '705.2' 5139.0 127.0]]


In [66]:
# Fixation
headers = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9']
fix_f = pd.read_csv('efix.csv', sep=',', skiprows = 1, header=None, names=headers, dtype={"col1": int, "col2": str, "col3": str, "col4": float, "col5": float, "col6": float, "col7": float, "col8": float, "col9": float})
fix_marker = fix_f.to_numpy()

# Blink
headers = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6']
blink_f = pd.read_csv('eblink.csv', sep=',', skiprows = 1, header=None, names=headers, dtype={"col1": int, "col2": str, "col3": str, "col4": float, "col5": float, "col6": float})
blink_marker = blink_f.to_numpy()

#sacc
headers = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10', 'col11', 'col12']
sacc_f = pd.read_csv('esacc.csv', sep=',', skiprows = 1, header=None, names=headers, dtype={"col1": int, "col2": str, "col3": str, "col4": float, "col5": float, "col6": float, "col7": float, "col8": float, "col9": float, "col10": float, "col11": float, "col12": float})
sacc_marker = sacc_f.to_numpy()
print(type(sacc_marker))
# for record in sacc_marker:
#     start_time = record[3]
#     end_time = record[4]
#     print(start_time,end_time)

<class 'numpy.ndarray'>


In [125]:
data_marker = np.empty((np.shape(data)[0], 2), dtype=object)
data_marker[:] = ""


In [126]:
fix_index = 0
blink_index = 0
sacc_index = 0

def update_index(current_time, fix_index, blink_index, sacc_index):
    current_fix = fix_marker[fix_index]
    current_blink = blink_marker[blink_index]
    current_sacc = sacc_marker[sacc_index]
    if current_fix[4] < current_time:
        fix_index += 1
    if current_blink[4] < current_time:
        blink_index += 1
    if current_sacc[4] < current_time:
        sacc_index += 1


for i in range(np.shape(data)[0]):
    record = data[i]
    current_time = record[1]
    if fix_index < len(fix_marker) and fix_marker[fix_index][4] < current_time:
        fix_index += 1
    if blink_index < len(blink_marker) and blink_marker[blink_index][4] < current_time:
        blink_index += 1
    if sacc_index < len(sacc_marker) and sacc_marker[sacc_index][4] < current_time:
        sacc_index += 1

    if fix_index < len(fix_marker) and current_time >= fix_marker[fix_index][3] and current_time <= fix_marker[fix_index][4]:
        data_marker[i] = ["fix", fix_marker[fix_index][0]]
    elif blink_index < len(blink_marker) and current_time >= blink_marker[blink_index][3] and current_time <= blink_marker[blink_index][4]:
        data_marker[i] = ["blink", blink_marker[blink_index][0]]
    elif sacc_index < len(sacc_marker) and current_time >= sacc_marker[sacc_index][3] and current_time <= sacc_marker[sacc_index][4]:
        data_marker[i] = ["sacc", sacc_marker[sacc_index][0]]
    else:
        continue

In [127]:
data_marker

array([['', ''],
       ['', ''],
       ['', ''],
       ...,
       ['fix', 917],
       ['fix', 917],
       ['', '']], dtype=object)

In [129]:
new_data = np.concatenate((data, data_marker), axis = 1)
new_data

array([[0, 333603557.0, '1146.1', ..., 127.0, '', ''],
       [1, 333603557.0, '1145.8', ..., 127.0, '', ''],
       [2, 333603558.0, '1145.8', ..., 127.0, '', ''],
       ...,
       [744956, 333976193.0, '1234.4', ..., 127.0, 'fix', 917],
       [744957, 333976193.0, '1234.1', ..., 127.0, 'fix', 917],
       [744958, 333976194.0, '1233.8', ..., 127.0, '', '']], dtype=object)

In [132]:
pd.DataFrame(new_data).to_csv('match_data.csv', index = False, header  = ['Index', 'Time','x','y','pupil size','CR', 'event marker', 'event index'])

In [27]:
asc_file = r'./J030701.asc'
raw = open(asc_file)
# _ = pd.read_csv(r'D:\Jupiter_eyemovement_data\Pic_TOJ\formal\SESSION1\ASC\J030705.csv')

# #for line in data:
# #    n = 0 
# #    print(line)
data = _[['Time','x','y','pupil size','CR']]#[0:3]
data

def is_dig(char):
    """判断是否包含空格"""

    if type(char)==float:
        return True
    elif char.isdigit() == True:
        return True
    else:
        return False
    
final_data = pd.DataFrame()
event_list = [0]
df = data[['Time','x','y','pupil size','CR']]
df
#df['Time'] = str(df['Time'])
for i in df.index:
    #print(,type(int(data['Time'].loc[i])))
    z = 0
    if i % 100000 == 0:
        print('s')
    if is_dig(df['Time'].loc[i]) == False:
    
    #if is_space(df['Time'].loc[i]):
        #print('s')
        event_list.append(i)  
        event_data = df[event_list[-2]:event_list[-1]]
        
        event_data['event']= df['Time'].loc[i]
        final_data = final_data.append(event_data)
    
    final_data.to_csv(r"D:\Jupiter_eyemovement_data\Pic_TOJ\formal\SESSION1\ASC\J030705_testtest.csv")

s


C:\Users\SEANZH~1\AppData\Local\Temp/ipykernel_22772/396348675.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_data['event']= df['Time'].loc[i]


s
s
s
s
s
s


In [75]:
import pandas as pd

#asc_file = r'D:\Jupiter_eyemovement_data\Pic_TOJ\formal\SESSION1\ASC\J030705.asc' # 

#with open(asc_file) as f:
#    pass

### 首先把.asc 文件保存成.csv (具体格式要修改一下：1.删掉开头的信息  2. 改掉部分的内容。 注：以这个csv 文件为模板：J030705-csved； 如果你要批量进行操作，你可以尝试用代码实现下)
### 修改下面的 csv 文件路径 （之后你可以写成批处理文件，批量处理csv 文件）
### 读出来的最终csv 参考 J030705_test_v3.csv； 里面 其实还有一些需要处理，比如说，在MSG 文件中， 里面 是一个混合的字符，其中包含了一些信息，你可以尝试用.split()  来对这一列进行操作


data = pd.read_csv(r'D:\Jupiter_eyemovement_data\Pic_TOJ\formal\SESSION1\ASC\J030705-csved.csv', dtype={'Time': str})# 这里是你要导入的csv 文件路径

def is_dig(char): # 一些正则
    """判断是否为纯数字"""
    return str(char).isdigit()

c = pd.DataFrame()
event_list = [0]
_ = data[['Time','x','y','pupil size','CR']]#[0:1000]

for i in range(len(_)):
    if i % 100000 == 0:
        print('s')

    if not is_dig(_['Time'][i]):
        event_list.append(i)
        event_data = _[event_list[-2]:event_list[-1]]
        event_data['event'] = _['Time'][i]
        c = c.append(event_data)


final_data = pd.DataFrame()
event_list = [0]
df = c[['Time','x','y','pupil size','CR','event']]#[0:1000]
for i in range(len(df)):
    if i % 100000 == 0:
        print('s')

    if df['Time'][i] == 'MSG':
    #@if df['Time'][i] == 'MSG':
        event_list.append(i)
        event_data = df[event_list[-2]:event_list[-1]]
        event_data['MSG'] = df['x'][i]
        final_data = final_data.append(event_data)

final_data['event_final'] = final_data['event'].str.split().str.get(0)
final_data.to_csv(r"D:\Jupiter_eyemovement_data\Pic_TOJ\formal\SESSION1\ASC\J030705_test_v3.csv") # 这是保存出来的路径


s


C:\Users\SEANZH~1\AppData\Local\Temp/ipykernel_22772/2956313283.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_data['event'] = _['Time'][i]


s
s
s
s
s
s
s


C:\Users\SEANZH~1\AppData\Local\Temp/ipykernel_22772/2956313283.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_data['MSG'] = df['x'][i]


s
s
s
s
s
s


In [77]:
x = 'SFIX R   335349005'
x.split(' ')[0]

'SFIX'

In [72]:
df

,Time,x,y,pupil size,CR,event
0,335348998,1278.5,467.4,3892.0,127.0,SFIX R 335349005
1,335348998,1278,468,3883.0,127.0,SFIX R 335349005
2,335348999,1277.5,469.3,3878.0,127.0,SFIX R 335349005
3,335348999,1277.5,470.4,3882.0,127.0,SFIX R 335349005
4,335349000,1276.9,471.2,3891.0,127.0,SFIX R 335349005
...,...,...,...,...,...,...
865,335349426,1273.2,690.3,4027.0,127.0,ESACC R 335349388
866,335349426,1272.7,690.3,4029.0,127.0,ESACC R 335349388
867,335349427,1272.1,690.2,4028.0,127.0,ESACC R 335349388
868,ESACC R 335349388,335349427,39,1138.5,469.0,SFIX R 335349428


In [53]:
df['Time'][i]

'END'

In [54]:
event_data[]

,Time,x,y,pupil size,CR,event
662336,INPUT,335678566,127,NaN,NaN,END


In [31]:
final_data.to_csv(r"D:\Jupiter_eyemovement_data\Pic_TOJ\formal\SESSION1\ASC\J030705_testtest.csv")

In [23]:
final_data

,Time,x,y,pupil size,CR,event
0,** CONVERTED FROM D:\Jupiter_eyemovement_data\...,NaN,NaN,NaN,NaN,** DATE: Mon Mar 6 17:24:00 2023
1,** DATE: Mon Mar 6 17:24:00 2023,NaN,NaN,NaN,NaN,** TYPE: EDF_FILE BINARY EVENT SAMPLE TAGGED
2,** TYPE: EDF_FILE BINARY EVENT SAMPLE TAGGED,NaN,NaN,NaN,NaN,** VERSION: EYELINK II 1
3,** VERSION: EYELINK II 1,NaN,NaN,NaN,NaN,** SOURCE: EYELINK CL
4,** SOURCE: EYELINK CL,NaN,NaN,NaN,NaN,** EYELINK II CL v5.15 Jan 24 2018
...,...,...,...,...,...,...
190873,335443982\t 1388.8\t 665.9\t 4799.0\t 127.0\...,NaN,NaN,NaN,NaN,335443982\t 1389.0\t 664.5\t 4805.0\t 127.0\...
190874,335443982\t 1389.0\t 664.5\t 4805.0\t 127.0\...,NaN,NaN,NaN,NaN,335443983\t 1389.1\t 663.4\t 4803.0\t 127.0\...
190875,335443983\t 1389.1\t 663.4\t 4803.0\t 127.0\...,NaN,NaN,NaN,NaN,335443983\t 1389.0\t 662.9\t 4793.0\t 127.0\...
190876,335443983\t 1389.0\t 662.9\t 4793.0\t 127.0\...,NaN,NaN,NaN,NaN,335443984\t 1388.8\t 660.5\t 4787.0\t 127.0\...


# JIanhua coding

In [24]:
asc = open(asc_file)

fix_x_encoding_0 = [] # List of all x positions
fix_y_encoding_0 = [] # List of all y positions
fix_duration_encoding_0 = [] # List of all durations

SACC_start_x_encoding_0 = []
SACC_start_y_encoding_0 = []
SACC_duration_encoding_0 = []
SACC_end_x_encoding_0 = []
SACC_end_y_encoding_0 = []

BLINK_duration_encoding_0 = []



new_trial = False
for line in asc:
    # Convert the current data line into a list
    tmp_data = line.rstrip().split()
    # Get the correct screen resolution from the DISPLAY_COORDS message
    # MSG    4302897 DISPLAY_COORDS 0 0 2559 1439
    if 'DISPLAY_COORDS' in line: 
        scn_w = int(tmp_data[-2]) + 1 
        scn_h = int(tmp_data[-1]) + 1
    # Look for the message marking image onset 
    #memory stage-image
    if 'forcedmemory_start_pic_0' in line:
        trial_start = int(float(tmp_data[1]))
        new_trial = True
        # Store the position and duration of all fixations in lists
        fix_x_memory = []
        fix_y_memory = []
        fix_duration_memory = []
        
        SACC_start_x_memory = []
        SACC_start_y_memory = []
        SACC_duration_memory = []
        SACC_end_x_memory = []
        SACC_end_y_memory = []

        BLINK_duration_memory = []
    
    if 'forcedmemory_end_pic_0' in line:
        new_trial = False
        # Add the fixation data for this trial to the all-trials lists
        fix_x_encoding_0.extend(fix_x_memory)
        fix_y_encoding_0.extend(fix_y_memory)
        fix_duration_encoding_0.extend(fix_duration_memory)
        
        SACC_start_x_encoding_0.extend(SACC_start_x_memory)
        SACC_start_y_encoding_0.extend(SACC_start_y_memory)
        SACC_duration_encoding_0.extend(SACC_duration_memory)
        SACC_end_x_encoding_0.extend(SACC_end_x_memory)
        SACC_end_y_encoding_0.extend(SACC_end_y_memory)
        
        BLINK_duration_encoding_0.extend(BLINK_duration_memory)
        
        
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'EFIX' in line:
            # EFIX R   349482768	349482935	167	 1312.5	  729.7	   3770
            start_time, end_time,duration_e_f, x, y = [int(float(x)) for x in tmp_data[2:7]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                duration_e_f = duration_e_f - (trial_start - start_time)
                start_time = trial_start
            fix_x_encoding_0.append(x)
            fix_y_encoding_0.append(y)
            fix_duration_encoding_0.append(duration_e_f)  
            
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'ESACC' in line:
            #ESACC R  554166	554204	38	 1207.8	  978.0	 1293.4	  761.3	   8.32	amplitude    517 velocity

            start_time, end_time, duration_e_s, x1, y1,x2,y2 = [int(float(x)) for x in tmp_data[2:9]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                duration_e_s = duration_e_s - (trial_start - start_time)
                start_time = trial_start
            SACC_start_x_encoding_0.append(x1)
            SACC_start_y_encoding_0.append(y1)
            SACC_duration_encoding_0.append(duration_e_s)
            SACC_end_x_encoding_0.append(x2)
            SACC_end_y_encoding_0.append(y2)       
        
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
            if 'EBLINK' in line:
            #EBLINK R 565909	565986	77

                start_time, end_time,duration_e_b = [int(float(x)) for x in tmp_data[2:]]
                if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_e_b = duration_e_b - (trial_start - start_time)
                    start_time = trial_start 
                #duration_e_b = [str(float(x)) for x in tmp_data[4:]]
                BLINK_duration_encoding_0.append(duration_e_b)
asc.close()  
# Combine the fixation data lists into a dictionary
eye_data_encoding_pic = {
    'fix_x_e_position': fix_x_encoding_0,
    'fix_y_e_position': fix_y_encoding_0,
    'fix_e_duration': fix_duration_encoding_0,
    'SACC_x_e_s_position': SACC_start_x_encoding_0,
    'SACC_y_e_s_position': SACC_start_y_encoding_0,
    'SACC_e_duration': SACC_duration_encoding_0,
    'SACC_x_e_e_position': SACC_end_x_encoding_0,
    'SACC_y_e_e_position': SACC_end_y_encoding_0, 
    'BLINK_e_duration': BLINK_duration_encoding_0, 
}               
                
df_data_encoding_pic_0 = pd.DataFrame.from_dict(eye_data_encoding_pic, orient='index')
print(df_data_encoding_pic_0.T)  

FileNotFoundError: [Errno 2] No such file or directory: '/Users/apple/work/lab/eye/J030705.asc'

In [1287]:
asc = open(asc_file)

fix_x_encoding_1 = [] # List of all x positions
fix_y_encoding_1 = [] # List of all y positions
fix_duration_encoding_1 = [] # List of all durations

SACC_start_x_encoding_1 = []
SACC_start_y_encoding_1 = []
SACC_duration_encoding_1 = []
SACC_end_x_encoding_1 = []
SACC_end_y_encoding_1 = []

BLINK_duration_encoding_1 = []



new_trial = False
for line in asc:
    # Convert the current data line into a list
    tmp_data = line.rstrip().split()
    # Get the correct screen resolution from the DISPLAY_COORDS message
    # MSG    4302897 DISPLAY_COORDS 0 0 2559 1439
    if 'DISPLAY_COORDS' in line: 
        scn_w = int(tmp_data[-2]) + 1 
        scn_h = int(tmp_data[-1]) + 1
    # Look for the message marking image onset 
    #memory stage-image
    if 'forcedmemory_start_pic_1' in line:
        trial_start = int(float(tmp_data[1]))
        new_trial = True
        # Store the position and duration of all fixations in lists
        fix_x_memory = []
        fix_y_memory = []
        fix_duration_memory = []
        
        SACC_start_x_memory = []
        SACC_start_y_memory = []
        SACC_duration_memory = []
        SACC_end_x_memory = []
        SACC_end_y_memory = []

        BLINK_duration_memory = []
    
    if 'forcedmemory_end_pic_1' in line:
        new_trial = False
        # Add the fixation data for this trial to the all-trials lists
        fix_x_encoding_1.extend(fix_x_memory)
        fix_y_encoding_1.extend(fix_y_memory)
        fix_duration_encoding_1.extend(fix_duration_memory)
        
        SACC_start_x_encoding_1.extend(SACC_start_x_memory)
        SACC_start_y_encoding_1.extend(SACC_start_y_memory)
        SACC_duration_encoding_1.extend(SACC_duration_memory)
        SACC_end_x_encoding_1.extend(SACC_end_x_memory)
        SACC_end_y_encoding_1.extend(SACC_end_y_memory)
        
        BLINK_duration_encoding_1.extend(BLINK_duration_memory)
        
        
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'EFIX' in line:
            # EFIX R   349482768	349482935	167	 1312.5	  729.7	   3770
            start_time, end_time,duration_e_f, x, y = [int(float(x)) for x in tmp_data[2:7]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                duration_e_f = duration_e_f - (trial_start - start_time)
                start_time = trial_start
            fix_x_encoding_1.append(x)
            fix_y_encoding_1.append(y)
            fix_duration_encoding_1.append(duration_e_f)  
            
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'ESACC' in line:
            #ESACC R  554166	554204	38	 1207.8	  978.0	 1293.4	  761.3	   8.32	amplitude    517 velocity

            start_time, end_time, duration_e_s, x1, y1,x2,y2 = [int(float(x)) for x in tmp_data[2:9]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                duration_e_s = duration_e_s - (trial_start - start_time)
                start_time = trial_start
            SACC_start_x_encoding_1.append(x1)
            SACC_start_y_encoding_1.append(y1)
            SACC_duration_encoding_1.append(duration_e_s)
            SACC_end_x_encoding_1.append(x2)
            SACC_end_y_encoding_1.append(y2)       
        
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
            if 'EBLINK' in line:
            #EBLINK R 565909	565986	77

                start_time, end_time,duration_e_b = [int(float(x)) for x in tmp_data[2:]]
                if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_e_b = duration_e_b - (trial_start - start_time)
                    start_time = trial_start 
                #duration_e_b = [str(float(x)) for x in tmp_data[4:]]
                BLINK_duration_encoding_1.append(duration_e_b)
asc.close()  
# Combine the fixation data lists into a dictionary
eye_data_encoding_pic = {
    'fix_x_e_position': fix_x_encoding_1,
    'fix_y_e_position': fix_y_encoding_1,
    'fix_e_duration': fix_duration_encoding_1,
    'SACC_x_e_s_position': SACC_start_x_encoding_1,
    'SACC_y_e_s_position': SACC_start_y_encoding_1,
    'SACC_e_duration': SACC_duration_encoding_1,
    'SACC_x_e_e_position': SACC_end_x_encoding_1,
    'SACC_y_e_e_position': SACC_end_y_encoding_1, 
    'BLINK_e_duration': BLINK_duration_encoding_1, 
}               
                
df_data_encoding_pic_1 = pd.DataFrame.from_dict(eye_data_encoding_pic, orient='index')
print(df_data_encoding_pic_1.T)               
                

     fix_x_e_position  fix_y_e_position  fix_e_duration  SACC_x_e_s_position  \
0              2362.0             982.0           419.0               2343.0   
1              1272.0             631.0           273.0               1274.0   
2              1379.0             668.0           247.0               1387.0   
3              2368.0             862.0           944.0               2380.0   
4              1845.0             321.0           564.0               1838.0   
..                ...               ...             ...                  ...   
174            1245.0             751.0           274.0               1927.0   
175            1907.0             -57.0           550.0               1751.0   
176            1754.0             326.0           385.0               1180.0   
177            1185.0             213.0           295.0               2077.0   
178            2069.0             988.0           366.0                  NaN   

     SACC_y_e_s_position  SACC_e_durati

In [1288]:
asc = open(asc_file)

fix_x_encoding_2 = [] # List of all x positions
fix_y_encoding_2 = [] # List of all y positions
fix_duration_encoding_2 = [] # List of all durations

SACC_start_x_encoding_2 = []
SACC_start_y_encoding_2 = []
SACC_duration_encoding_2 = []
SACC_end_x_encoding_2 = []
SACC_end_y_encoding_2 = []

BLINK_duration_encoding_2 = []



new_trial = False
for line in asc:
    # Convert the current data line into a list
    tmp_data = line.rstrip().split()
    # Get the correct screen resolution from the DISPLAY_COORDS message
    # MSG    4302897 DISPLAY_COORDS 0 0 2559 1439
    if 'DISPLAY_COORDS' in line: 
        scn_w = int(tmp_data[-2]) + 1 
        scn_h = int(tmp_data[-1]) + 1
    # Look for the message marking image onset 
    #memory stage-image
    if 'forcedmemory_start_pic_2' in line:
        trial_start = int(float(tmp_data[1]))
        new_trial = True
        # Store the position and duration of all fixations in lists
        fix_x_memory = []
        fix_y_memory = []
        fix_duration_memory = []
        
        SACC_start_x_memory = []
        SACC_start_y_memory = []
        SACC_duration_memory = []
        SACC_end_x_memory = []
        SACC_end_y_memory = []

        BLINK_duration_memory = []
    
    if 'forcedmemory_end_pic_2' in line:
        new_trial = False
        # Add the fixation data for this trial to the all-trials lists
        fix_x_encoding_2.extend(fix_x_memory)
        fix_y_encoding_2.extend(fix_y_memory)
        fix_duration_encoding_2.extend(fix_duration_memory)
        
        SACC_start_x_encoding_2.extend(SACC_start_x_memory)
        SACC_start_y_encoding_2.extend(SACC_start_y_memory)
        SACC_duration_encoding_2.extend(SACC_duration_memory)
        SACC_end_x_encoding_2.extend(SACC_end_x_memory)
        SACC_end_y_encoding_2.extend(SACC_end_y_memory)
        
        BLINK_duration_encoding_2.extend(BLINK_duration_memory)
        
        
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'EFIX' in line:
            # EFIX R   349482768	349482935	167	 1312.5	  729.7	   3770
            start_time, end_time,duration_e_f, x, y = [int(float(x)) for x in tmp_data[2:7]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                duration_e_f = duration_e_f - (trial_start - start_time)
                start_time = trial_start
            fix_x_encoding_2.append(x)
            fix_y_encoding_2.append(y)
            fix_duration_encoding_2.append(duration_e_f)  
            
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'ESACC' in line:
            #ESACC R  554166	554204	38	 1207.8	  978.0	 1293.4	  761.3	   8.32	amplitude    517 velocity

            start_time, end_time, duration_e_s, x1, y1,x2,y2 = [int(float(x)) for x in tmp_data[2:9]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                duration_e_s = duration_e_s - (trial_start - start_time)
                start_time = trial_start
            SACC_start_x_encoding_2.append(x1)
            SACC_start_y_encoding_2.append(y1)
            SACC_duration_encoding_2.append(duration_e_s)
            SACC_end_x_encoding_2.append(x2)
            SACC_end_y_encoding_2.append(y2)       
        
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
            if 'EBLINK' in line:
            #EBLINK R 565909	565986	77

                start_time, end_time,duration_e_b = [int(float(x)) for x in tmp_data[2:]]
                if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_e_b = duration_e_b - (trial_start - start_time)
                    start_time = trial_start 
                #duration_e_b = [str(float(x)) for x in tmp_data[4:]]
                BLINK_duration_encoding_2.append(duration_e_b)
asc.close()  
# Combine the fixation data lists into a dictionary
eye_data_encoding_pic = {
    'fix_x_e_position': fix_x_encoding_2,
    'fix_y_e_position': fix_y_encoding_2,
    'fix_e_duration': fix_duration_encoding_2,
    'SACC_x_e_s_position': SACC_start_x_encoding_2,
    'SACC_y_e_s_position': SACC_start_y_encoding_2,
    'SACC_e_duration': SACC_duration_encoding_2,
    'SACC_x_e_e_position': SACC_end_x_encoding_2,
    'SACC_y_e_e_position': SACC_end_y_encoding_2, 
    'BLINK_e_duration': BLINK_duration_encoding_2, 
}               
                
df_data_encoding_pic_2 = pd.DataFrame.from_dict(eye_data_encoding_pic, orient='index')
print(df_data_encoding_pic_2.T)               
                

    fix_x_e_position  fix_y_e_position  fix_e_duration  SACC_x_e_s_position  \
0             2272.0             678.0          1086.0               2266.0   
1             1218.0             754.0           230.0               1202.0   
2             1098.0             645.0           438.0               1085.0   
3               22.0             882.0           197.0                 32.0   
4             2313.0             757.0          1455.0               2319.0   
5             1407.0             813.0           279.0               2392.0   
6             1197.0             789.0           454.0               1399.0   
7             1347.0             874.0           271.0               1187.0   
8             2310.0             672.0          1492.0               1341.0   
9             2625.0            1818.0           149.0               2348.0   
10            1263.0             649.0           226.0               2627.0   
11            1303.0             659.0           465

In [1289]:
asc = open(asc_file)

fix_x_encoding_3 = [] # List of all x positions
fix_y_encoding_3 = [] # List of all y positions
fix_duration_encoding_3 = [] # List of all durations

SACC_start_x_encoding_3 = []
SACC_start_y_encoding_3 = []
SACC_duration_encoding_3 = []
SACC_end_x_encoding_3 = []
SACC_end_y_encoding_3 = []

BLINK_duration_encoding_3 = []



new_trial = False
for line in asc:
    # Convert the current data line into a list
    tmp_data = line.rstrip().split()
    # Get the correct screen resolution from the DISPLAY_COORDS message
    # MSG    4302897 DISPLAY_COORDS 0 0 2559 1439
    if 'DISPLAY_COORDS' in line: 
        scn_w = int(tmp_data[-2]) + 1 
        scn_h = int(tmp_data[-1]) + 1
    # Look for the message marking image onset 
    #memory stage-image
    if 'forcedmemory_start_pic_3' in line:
        trial_start = int(float(tmp_data[1]))
        new_trial = True
        # Store the position and duration of all fixations in lists
        fix_x_memory = []
        fix_y_memory = []
        fix_duration_memory = []
        
        SACC_start_x_memory = []
        SACC_start_y_memory = []
        SACC_duration_memory = []
        SACC_end_x_memory = []
        SACC_end_y_memory = []

        BLINK_duration_memory = []
    
    if 'forcedmemory_end_pic_3' in line:
        new_trial = False
        # Add the fixation data for this trial to the all-trials lists
        fix_x_encoding_3.extend(fix_x_memory)
        fix_y_encoding_3.extend(fix_y_memory)
        fix_duration_encoding_3.extend(fix_duration_memory)
        
        SACC_start_x_encoding_3.extend(SACC_start_x_memory)
        SACC_start_y_encoding_3.extend(SACC_start_y_memory)
        SACC_duration_encoding_3.extend(SACC_duration_memory)
        SACC_end_x_encoding_3.extend(SACC_end_x_memory)
        SACC_end_y_encoding_3.extend(SACC_end_y_memory)
        
        BLINK_duration_encoding_3.extend(BLINK_duration_memory)
        
        
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'EFIX' in line:
            # EFIX R   349482768	349482935	167	 1312.5	  729.7	   3770
            start_time, end_time,duration_e_f, x, y = [int(float(x)) for x in tmp_data[2:7]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                duration_e_f = duration_e_f - (trial_start - start_time)
                start_time = trial_start
            fix_x_encoding_3.append(x)
            fix_y_encoding_3.append(y)
            fix_duration_encoding_3.append(duration_e_f)  
            
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'ESACC' in line:
            #ESACC R  554166	554204	38	 1207.8	  978.0	 1293.4	  761.3	   8.32	amplitude    517 velocity

            start_time, end_time, duration_e_s, x1, y1,x2,y2 = [int(float(x)) for x in tmp_data[2:9]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                duration_e_s = duration_e_s - (trial_start - start_time)
                start_time = trial_start
            SACC_start_x_encoding_3.append(x1)
            SACC_start_y_encoding_3.append(y1)
            SACC_duration_encoding_3.append(duration_e_s)
            SACC_end_x_encoding_3.append(x2)
            SACC_end_y_encoding_3.append(y2)       
        
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
            if 'EBLINK' in line:
            #EBLINK R 565909	565986	77

                start_time, end_time,duration_e_b = [int(float(x)) for x in tmp_data[2:]]
                if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_e_b = duration_e_b - (trial_start - start_time)
                    start_time = trial_start 
                #duration_e_b = [str(float(x)) for x in tmp_data[4:]]
                BLINK_duration_encoding_3.append(duration_e_b)
asc.close()  
# Combine the fixation data lists into a dictionary
eye_data_encoding_pic = {
    'fix_x_e_position': fix_x_encoding_3,
    'fix_y_e_position': fix_y_encoding_3,
    'fix_e_duration': fix_duration_encoding_3,
    'SACC_x_e_s_position': SACC_start_x_encoding_3,
    'SACC_y_e_s_position': SACC_start_y_encoding_3,
    'SACC_e_duration': SACC_duration_encoding_3,
    'SACC_x_e_e_position': SACC_end_x_encoding_3,
    'SACC_y_e_e_position': SACC_end_y_encoding_3, 
    'BLINK_e_duration': BLINK_duration_encoding_3, 
}               
                
df_data_encoding_pic_3 = pd.DataFrame.from_dict(eye_data_encoding_pic, orient='index')
print(df_data_encoding_pic_3.T)               
                

     fix_x_e_position  fix_y_e_position  fix_e_duration  SACC_x_e_s_position  \
0              2491.0            1160.0           226.0               2469.0   
1              1484.0             705.0           494.0               1476.0   
2              1516.0             743.0           756.0               1522.0   
3              2384.0            1129.0           284.0               2386.0   
4              1471.0             756.0           248.0               1472.0   
..                ...               ...             ...                  ...   
152            2427.0             748.0          1899.0               2048.0   
153            2061.0             235.0           390.0               2520.0   
154            2512.0             616.0          1040.0               2759.0   
155            2760.0               4.0            39.0               2747.0   
156            2760.0               5.0           371.0                  NaN   

     SACC_y_e_s_position  SACC_e_durati

In [1290]:
df_encoding = pd.concat([df_data_encoding_pic_0.T, df_data_encoding_pic_1.T, df_data_encoding_pic_2.T, df_data_encoding_pic_3.T]).reset_index()
df_encoding
 

,index,fix_x_e_position,fix_y_e_position,fix_e_duration,SACC_x_e_s_position,SACC_y_e_s_position,SACC_e_duration,SACC_x_e_e_position,SACC_y_e_e_position,BLINK_e_duration
0,0,1247.0,703.0,216.0,1245.0,714.0,27.0,1318.0,651.0,93.0
1,1,1319.0,644.0,389.0,1317.0,645.0,43.0,1288.0,834.0,285.0
2,2,1289.0,836.0,284.0,1291.0,837.0,171.0,2459.0,1060.0,109.0
3,3,2460.0,1052.0,861.0,2475.0,1029.0,38.0,2527.0,860.0,104.0
4,4,2546.0,844.0,413.0,2549.0,846.0,53.0,2413.0,1407.0,NaN
...,...,...,...,...,...,...,...,...,...,...
400,152,2427.0,748.0,1899.0,2048.0,232.0,86.0,2503.0,616.0,NaN
401,153,2061.0,235.0,390.0,2520.0,612.0,68.0,2751.0,14.0,NaN
402,154,2512.0,616.0,1040.0,2759.0,-4.0,24.0,2769.0,-9.0,NaN
403,155,2760.0,4.0,39.0,2747.0,24.0,39.0,2744.0,21.0,NaN


In [1291]:
df_eye = {'fix_e_duration':[], 'fix_x_e':[], 'fix_y_e':[],
         'SACC_e_duration':[], 
          'BLINK_e_duration':[], 
          'region_e':[],
         }

for i in range(0,len(df_encoding)):
    df_eye['fix_e_duration'].append(df_encoding['fix_e_duration'][i])
    df_eye['fix_x_e'].append(df_encoding['fix_x_e_position'][i])
    df_eye['fix_y_e'].append(df_encoding['fix_y_e_position'][i])
    if 992< df_encoding['fix_x_e_position'][i] <1568 and 560< df_encoding['fix_y_e_position'][i] <880:
        df_eye['region_e'].append('on_pic_e')
    elif df_encoding['fix_x_e_position'][i] <2560 and df_encoding['fix_y_e_position'][i] <1440:
        df_eye['region_e'].append('off_pic_e')
    else:
        df_eye['region_e'].append('N/A')
            
    df_eye['SACC_e_duration'].append(df_encoding['SACC_e_duration'][i])    
    df_eye['BLINK_e_duration'].append(df_encoding['BLINK_e_duration'][i])
          
df_eye = pd.DataFrame.from_dict(df_eye, orient='index')
print(df_eye.T)
df_eye = df_eye.T
print(df_eye)

    fix_e_duration fix_x_e fix_y_e SACC_e_duration BLINK_e_duration   region_e
0            216.0  1247.0   703.0            27.0             93.0   on_pic_e
1            389.0  1319.0   644.0            43.0            285.0   on_pic_e
2            284.0  1289.0   836.0           171.0            109.0   on_pic_e
3            861.0  2460.0  1052.0            38.0            104.0  off_pic_e
4            413.0  2546.0   844.0            53.0              NaN  off_pic_e
..             ...     ...     ...             ...              ...        ...
400         1899.0  2427.0   748.0            86.0              NaN  off_pic_e
401          390.0  2061.0   235.0            68.0              NaN  off_pic_e
402         1040.0  2512.0   616.0            24.0              NaN  off_pic_e
403           39.0  2760.0     4.0            39.0              NaN        N/A
404          371.0  2760.0     5.0             NaN              NaN        N/A

[405 rows x 6 columns]
    fix_e_duration fix_x_e f

In [1292]:
#df_eye.to_csv('df_eye_encoding_030705.csv')

fix_e_on = df_eye[df_eye['region_e']=='on_pic_e']['fix_e_duration'].sum()
fix_e_off = df_eye[df_eye['region_e']=='off_pic_e']['fix_e_duration'].sum()
fix_e_out = df_eye[df_eye['region_e']=='N/A']['fix_e_duration'].sum()
fix_e_valid = df_eye[df_eye['region_e']!='N/A']['fix_e_duration'].sum()
fix_e_total = df_eye['fix_e_duration'].sum()

SACC_e_total = df_eye['SACC_e_duration'].sum()
BLINK_e_total = df_eye['BLINK_e_duration'].sum()
#eye_e_total = fix_e_total+SACC_e_total+BLINK_e_total
eye_e_total = fix_e_total+SACC_e_total
percent_fix_e_on_pic = fix_e_on/fix_e_valid
percent_fix_e_off_pic = fix_e_off/fix_e_valid

df_eye['percent_fix_e_on_pic'] = percent_fix_e_on_pic
df_eye['percent_fix_e_off_pic'] = percent_fix_e_off_pic
df_eye['fix_e_total'] = fix_e_total
df_eye['SACC_e_total'] = SACC_e_total
df_eye['BLINK_e_total'] = BLINK_e_total
df_eye['eye_e_total'] = eye_e_total
df_eye['fix_e_valid'] = fix_e_valid
df_eye['fix_e_on'] = fix_e_on
df_eye['fix_e_off'] = fix_e_off
df_eye['fix_e_out'] = fix_e_out


df_eye_encoding= df_eye
df_eye_encoding
#df_data_encoding_pic_0.T.to_csv(r'D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/ASC/df_eye_encoding_022302_1.csv')  

,fix_e_duration,fix_x_e,fix_y_e,SACC_e_duration,BLINK_e_duration,region_e,percent_fix_e_on_pic,percent_fix_e_off_pic,fix_e_total,SACC_e_total,BLINK_e_total,eye_e_total,fix_e_valid,fix_e_on,fix_e_off,fix_e_out
0,216.0,1247.0,703.0,27.0,93.0,on_pic_e,0.233685,0.766315,187606.0,36814.0,9997.0,224420.0,160618.0,37534.0,123084.0,26988.0
1,389.0,1319.0,644.0,43.0,285.0,on_pic_e,0.233685,0.766315,187606.0,36814.0,9997.0,224420.0,160618.0,37534.0,123084.0,26988.0
2,284.0,1289.0,836.0,171.0,109.0,on_pic_e,0.233685,0.766315,187606.0,36814.0,9997.0,224420.0,160618.0,37534.0,123084.0,26988.0
3,861.0,2460.0,1052.0,38.0,104.0,off_pic_e,0.233685,0.766315,187606.0,36814.0,9997.0,224420.0,160618.0,37534.0,123084.0,26988.0
4,413.0,2546.0,844.0,53.0,NaN,off_pic_e,0.233685,0.766315,187606.0,36814.0,9997.0,224420.0,160618.0,37534.0,123084.0,26988.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,1899.0,2427.0,748.0,86.0,NaN,off_pic_e,0.233685,0.766315,187606.0,36814.0,9997.0,224420.0,160618.0,37534.0,123084.0,26988.0
401,390.0,2061.0,235.0,68.0,NaN,off_pic_e,0.233685,0.766315,187606.0,36814.0,9997.0,224420.0,160618.0,37534.0,123084.0,26988.0
402,1040.0,2512.0,616.0,24.0,NaN,off_pic_e,0.233685,0.766315,187606.0,36814.0,9997.0,224420.0,160618.0,37534.0,123084.0,26988.0
403,39.0,2760.0,4.0,39.0,NaN,N/A,0.233685,0.766315,187606.0,36814.0,9997.0,224420.0,160618.0,37534.0,123084.0,26988.0


In [1293]:
print(fix_e_on, fix_e_off, fix_e_valid, fix_e_total, SACC_e_total, BLINK_e_total,)
print(percent_fix_e_on_pic, percent_fix_e_off_pic, SACC_e_total, BLINK_e_total)

37534.0 123084.0 160618.0 187606.0 36814.0 9997.0
0.23368489210424734 0.7663151078957526 36814.0 9997.0


In [1294]:
#3AFC stage  pic_0  
asc = open(asc_file)

fix_x_testing_0 = [] # List of all x positions
fix_y_testing_0 = [] # List of all y positions
fix_duration_testing_0 = [] # List of all durations

SACC_start_x_testing_0 = []
SACC_start_y_testing_0 = []
SACC_duration_testing_0 = []
SACC_end_x_testing_0 = []
SACC_end_y_testing_0 = []

BLINK_duration_testing_0 = []

new_trial = False
for line in asc:
    # Convert the current data line into a list
    tmp_data = line.rstrip().split()
    # Get the correct screen resolution from the DISPLAY_COORDS message
    # MSG    4302897 DISPLAY_COORDS 0 0 2559 1439
    if 'DISPLAY_COORDS' in line: 
        scn_w = int(tmp_data[-2]) + 1 
        scn_h = int(tmp_data[-1]) + 1
    # Look for the message marking image onset 
    #memory stage-image

    if 'forcedchoice_start_0' in line:
        trial_start = int(float(tmp_data[1]))
        new_trial = True
        # Store the position and duration of all fixations in lists
        fix_x_testing = []
        fix_y_testing = []
        fix_duration_testing = []
        
        SACC_start_x_testing = []
        SACC_start_y_testing = []
        SACC_duration_testing = []
        SACC_end_x_testing = []
        SACC_end_y_testing = []
        
        BLINK_duration_testing = []
        
    if 'forcedchoice_end_0' in line:
        new_trial = False
        # Add the fixation data for this trial to the all-trials lists
        fix_x_testing_0.extend(fix_x_testing)
        fix_y_testing_0.extend(fix_y_testing)
        fix_duration_testing_0.extend(fix_duration_testing)
        
        SACC_start_x_testing_0.extend(SACC_start_x_testing)
        SACC_start_y_testing_0.extend(SACC_start_y_testing)
        SACC_duration_testing_0.extend(SACC_duration_testing) 
        SACC_end_x_testing_0.extend(SACC_end_x_testing)
        SACC_end_y_testing_0.extend(SACC_end_y_testing)
        
        BLINK_duration_testing_0.extend(BLINK_duration_testing)
        
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'EFIX' in line:
            # EFIX R   349482768	349482935	167	 1312.5	  729.7	   3770
            start_time, end_time,duration_t_f_0, x6, y6 = [int(float(x)) for x in tmp_data[2:7]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_t_f_0 = duration_t_f_0 - (trial_start - start_time)
                    start_time = trial_start 
            fix_x_testing_0.append(x6)
            fix_y_testing_0.append(y6)
            fix_duration_testing_0.append(duration_t_f_0) 
            
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        #ESACC R  554166	554204	38	 1207.8	  978.0	 1293.4	  761.3	   8.32	amplitude    517 velocity
        if 'ESACC' in line:
            start_time, end_time,duration_t_s_0, x7, y7,x8,y8 = [int(float(x)) for x in tmp_data[2:9]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_t_s_0 = duration_t_s_0 - (trial_start - start_time)
                    start_time = trial_start 
            SACC_start_x_testing_0.append(x7)
            SACC_start_y_testing_0.append(y7)
            SACC_duration_testing_0.append(duration_t_s_0)
            SACC_end_x_testing_0.append(x8)
            SACC_end_y_testing_0.append(y8)
            
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'EBLINK' in line:
            #EBLINK R 565909	565986	77

            start_time, end_time,duration_t_0_b = [int(float(x)) for x in tmp_data[2:]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_t_0_b = duration_t_0_b - (trial_start - start_time)
                    start_time = trial_start
            
            BLINK_duration_testing_0.append(duration_t_0_b)
            
asc.close()
            # Combine the fixation data lists into a dictionary
eye_data_testing_pic_0 = {
    'fix_x_t_position': fix_x_testing_0,
    'fix_y_t_position': fix_y_testing_0,
    'fix_t_duration': fix_duration_testing_0,
    'SACC_x_t_s_position': SACC_start_x_testing_0,
    'SACC_y_t_s_position': SACC_start_y_testing_0,
    'SACC_t_duration': SACC_duration_testing_0,
    'SACC_x_t_e_position': SACC_end_x_testing_0,
    'SACC_y_t_e_position': SACC_end_y_testing_0,
    'BLINK_t_duration': BLINK_duration_testing_0,
}

# Create a DataFrame from the fixation data dictionary
#df_fixations_0 = pd.DataFrame(fixation_data)

#df = pd.DataFrame.from_dict(d, orient='index')

#print(df)
df_data_testing_pic_0 = pd.DataFrame.from_dict(eye_data_testing_pic_0, orient='index')
print(df_data_testing_pic_0.T)

df_data_testing_pic_0 =df_data_testing_pic_0.T
# Print the first 10 rows of the DataFrame
#df_fixations_0

    fix_x_t_position  fix_y_t_position  fix_t_duration  SACC_x_t_s_position  \
0             1259.0             720.0            51.0               1246.0   
1             1233.0             730.0           175.0               1231.0   
2              593.0             585.0           135.0                589.0   
3             1152.0             931.0           209.0               1148.0   
4             1104.0             919.0           309.0               1108.0   
5             1011.0             994.0           261.0               1008.0   
6              955.0            1087.0           413.0                963.0   
7              950.0            1153.0           165.0                943.0   
8             1224.0            1340.0           208.0               1225.0   
9             2627.0             512.0           162.0               2619.0   
10            1906.0             364.0           186.0               1904.0   
11            1933.0             577.0           158

In [1295]:
#3AFC stage pic_1   
asc = open(asc_file)

fix_x_testing_1 = [] # List of all x positions
fix_y_testing_1 = [] # List of all y positions
fix_duration_testing_1 = [] # List of all durations

SACC_start_x_testing_1 = []
SACC_start_y_testing_1 = []
SACC_duration_testing_1 = []
SACC_end_x_testing_1 = []
SACC_end_y_testing_1 = []

BLINK_duration_testing_1 = []

for line in asc:
    # Convert the current data line into a list
    tmp_data = line.rstrip().split()
    # Get the correct screen resolution from the DISPLAY_COORDS message
    # MSG    4302897 DISPLAY_COORDS 0 0 2559 1439
    if 'DISPLAY_COORDS' in line: 
        scn_w = int(tmp_data[-2]) + 1 
        scn_h = int(tmp_data[-1]) + 1
    # Look for the message marking image onset 
    if 'forcedchoice_start_1' in line:
        trial_start = int(float(tmp_data[1]))
        new_trial = True
        # Store the position and duration of all fixations in lists
        fix_x_testing = []
        fix_y_testing = []
        fix_duration_testing = []
        
        SACC_start_x_testing = []
        SACC_start_y_testing = []
        SACC_duration_testing = []
        SACC_end_x_testing = []
        SACC_end_y_testing = []
        
        BLINK_duration_testing = []
        
        
    if 'forcedchoice_end_1' in line:
        new_trial = False
        # Add the fixation data for this trial to the all-trials lists
        fix_x_testing_1.extend(fix_x_testing)
        fix_y_testing_1.extend(fix_y_testing)
        fix_duration_testing_1.extend(fix_duration_testing)
        
        SACC_start_x_testing_1.extend(SACC_start_x_testing)
        SACC_start_y_testing_1.extend(SACC_start_y_testing)
        SACC_duration_testing_1.extend(SACC_duration_testing) 
        SACC_end_x_testing_1.extend(SACC_end_x_testing)
        SACC_end_y_testing_1.extend(SACC_end_y_testing)
        
        BLINK_duration_testing_1.extend(BLINK_duration_testing)
    
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'EFIX' in line:
            # EFIX R   349482768	349482935	167	 1312.5	  729.7	   3770
            start_time, end_time,duration_t_f_1, x9, y9 = [int(float(x)) for x in tmp_data[2:7]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_t_f_1 = duration_t_f_1 - (trial_start - start_time)
                    start_time = trial_start
            fix_x_testing_1.append(x9)
            fix_y_testing_1.append(y9)
            fix_duration_testing_1.append(duration_t_f_1) 
            
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        #ESACC R  554166	554204	38	 1207.8	  978.0	 1293.4	  761.3	   8.32	amplitude    517 velocity
        if 'ESACC' in line:
            start_time, end_time, duration_t_s_1, x10, y10,x11,y11 = [int(float(x)) for x in tmp_data[2:9]]
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_t_s_1 = duration_t_s_1 - (trial_start - start_time)
                    start_time = trial_start 
            SACC_start_x_testing_1.append(x10)
            SACC_start_y_testing_1.append(y10)
            SACC_duration_testing_1.append(duration_t_s_1)
            SACC_end_x_testing_1.append(x11)
            SACC_end_y_testing_1.append(y11)       
            
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'EBLINK' in line:
            #EBLINK R 565909	565986	77

            start_time, end_time, duration_t_1_b = [int(float(x)) for x in tmp_data[2:]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_t_1_b = duration_t_1_b - (trial_start - start_time)
                    start_time = trial_start
            
            BLINK_duration_testing_1.append(duration_t_1_b)        
            
            
asc.close()
            # Combine the fixation data lists into a dictionary
eye_data_testing_pic_1 = {
    'fix_x_t_position': fix_x_testing_1,
    'fix_y_t_position': fix_y_testing_1,
    'fix_t_duration': fix_duration_testing_1,
    'SACC_x_t_s_position': SACC_start_x_testing_1,
    'SACC_y_t_s_position': SACC_start_y_testing_1,
    'SACC_t_duration': SACC_duration_testing_1,
    'SACC_x_t_e_position': SACC_end_x_testing_1,
    'SACC_y_t_e_position': SACC_end_y_testing_1,
    'BLINK_t_duration': BLINK_duration_testing_1,
}

# Create a DataFrame from the fixation data dictionary
df_data_testing_pic_1 = pd.DataFrame.from_dict(eye_data_testing_pic_1, orient='index')
print(df_data_testing_pic_1.T)
df_data_testing_pic_1 = df_data_testing_pic_1.T
# Print the first 10 rows of the DataFrame
#df_fixations_1

    fix_x_t_position  fix_y_t_position  fix_t_duration  SACC_x_t_s_position  \
0             1311.0             734.0           186.0               1305.0   
1             1253.0            1333.0           276.0               1243.0   
2             1184.0             114.0           173.0                250.0   
3              656.0             422.0           107.0               1175.0   
4              572.0             576.0           248.0                652.0   
5              565.0             607.0           294.0                572.0   
6              568.0             594.0          1518.0                565.0   
7              561.0             605.0          1451.0                566.0   
8             1271.0             708.0           164.0                567.0   
9             1258.0             944.0           217.0               1274.0   
10            1234.0            1350.0           443.0               1246.0   
11            1315.0             955.0           220

In [1296]:
#3AFC stage pic_2
asc = open(asc_file)

fix_x_testing_2 = [] # List of all x positions
fix_y_testing_2 = [] # List of all y positions
fix_duration_testing_2 = [] # List of all durations

SACC_start_x_testing_2 = []
SACC_start_y_testing_2 = []
SACC_duration_testing_2 = []
SACC_end_x_testing_2 = []
SACC_end_y_testing_2 = []

BLINK_duration_testing_2 = []

new_trial = False
for line in asc:
    # Convert the current data line into a list
    tmp_data = line.rstrip().split()
    # Get the correct screen resolution from the DISPLAY_COORDS message
    # MSG    4302897 DISPLAY_COORDS 0 0 2559 1439
    if 'DISPLAY_COORDS' in line: 
        scn_w = int(tmp_data[-2]) + 1 
        scn_h = int(tmp_data[-1]) + 1
    # Look for the message marking image onset 
    if 'forcedchoice_start_2' in line:
        trial_start = int(float(tmp_data[1]))
        new_trial = True
        # Store the position and duration of all fixations in lists
        fix_x_testing = []
        fix_y_testing = []
        fix_duration_testing = []
        
        SACC_start_x_testing = []
        SACC_start_y_testing = []
        SACC_duration_testing = []
        SACC_end_x_testing = []
        SACC_end_y_testing = []
        
        BLINK_duration_testing = []
        
    if 'forcedchoice_end_2' in line:
        new_trial = False
        # Add the fixation data for this trial to the all-trials lists
        fix_x_testing_2.extend(fix_x_testing)
        fix_y_testing_2.extend(fix_y_testing)
        fix_duration_testing_2.extend(fix_duration_testing)
        
        SACC_start_x_testing_2.extend(SACC_start_x_testing)
        SACC_start_y_testing_2.extend(SACC_start_y_testing)
        SACC_duration_testing_2.extend(SACC_duration_testing) 
        SACC_end_x_testing_2.extend(SACC_end_x_testing)
        SACC_end_y_testing_2.extend(SACC_end_y_testing)
        
        BLINK_duration_testing_2.extend(BLINK_duration_testing)
        
    
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'EFIX' in line:
            # EFIX R   349482768	349482935	167	 1312.5	  729.7	   3770
            start_time, end_time, duration_t_f_2, x12, y12 = [int(float(x)) for x in tmp_data[2:7]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_t_f_2 = duration_t_f_2 - (trial_start - start_time)
                    start_time = trial_start
            fix_x_testing_2.append(x12)
            fix_y_testing_2.append(y12)
            fix_duration_testing_2.append(duration_t_f_2) 
            
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        #ESACC R  554166	554204	38	 1207.8	  978.0	 1293.4	  761.3	   8.32	amplitude    517 velocity
        if 'ESACC' in line:
            start_time, end_time, duration_t_s_2, x13, y13,x14,y14 = [int(float(x)) for x in tmp_data[2:9]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_t_s_2 = duration_t_s_2 - (trial_start - start_time)
                    start_time = trial_start
            SACC_start_x_testing_2.append(x13)
            SACC_start_y_testing_2.append(y13)
            SACC_duration_testing_2.append(duration_t_s_2)
            SACC_end_x_testing_2.append(x14)
            SACC_end_y_testing_2.append(y14)
            
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'EBLINK' in line:
            #EBLINK R 565909	565986	77

            start_time, end_time, duration_t_2_b = [int(float(x)) for x in tmp_data[2:]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_t_2_b = duration_t_2_b - (trial_start - start_time)
                    start_time = trial_start
            
            BLINK_duration_testing_2.append(duration_t_2_b)
                      
asc.close()
            # Combine the fixation data lists into a dictionary
eye_data_testing_pic_2 = {
    'fix_x_t_position': fix_x_testing_2,
    'fix_y_t_position': fix_y_testing_2,
    'fix_t_duration': fix_duration_testing_2,
    'SACC_x_t_s_position': SACC_start_x_testing_2,
    'SACC_y_t_s_position': SACC_start_y_testing_2,
    'SACC_t_duration': SACC_duration_testing_2,
    'SACC_x_t_e_position': SACC_end_x_testing_2,
    'SACC_y_t_e_position': SACC_end_y_testing_2,
    'BLINK_t_duration': BLINK_duration_testing_2,
}

# Create a DataFrame from the fixation data dictionary
#df_fixations_0 = pd.DataFrame(fixation_data)

#df = pd.DataFrame.from_dict(d, orient='index')

#print(df)
df_data_testing_pic_2 = pd.DataFrame.from_dict(eye_data_testing_pic_2, orient='index')
print(df_data_testing_pic_2.T)
df_data_testing_pic_2 =df_data_testing_pic_2.T
# Print the first 10 rows of the DataFrame
#df_fixations_2

     fix_x_t_position  fix_y_t_position  fix_t_duration  SACC_x_t_s_position  \
0              1895.0            1334.0            61.0               1255.0   
1              1281.0             958.0           197.0               1891.0   
2              1236.0            1356.0           206.0               1272.0   
3              1060.0             875.0           448.0               1237.0   
4               969.0             873.0           334.0               1060.0   
..                ...               ...             ...                  ...   
171             624.0             588.0           230.0                605.0   
172             775.0             218.0           204.0                770.0   
173             681.0             201.0           270.0                682.0   
174             680.0             157.0           376.0                682.0   
175            1842.0             308.0           156.0               1838.0   

     SACC_y_t_s_position  SACC_t_durati

In [1297]:
#3AFC stage pic_3

asc = open(asc_file)

fix_x_testing_3 = [] # List of all x positions
fix_y_testing_3 = [] # List of all y positions
fix_duration_testing_3 = [] # List of all durations

SACC_start_x_testing_3 = []
SACC_start_y_testing_3 = []
SACC_duration_testing_3 = []
SACC_end_x_testing_3 = []
SACC_end_y_testing_3 = []

BLINK_duration_testing_3 = []

for line in asc:
    # Convert the current data line into a list
    tmp_data = line.rstrip().split()
    # Get the correct screen resolution from the DISPLAY_COORDS message
    # MSG    4302897 DISPLAY_COORDS 0 0 2559 1439
    if 'DISPLAY_COORDS' in line: 
        scn_w = int(tmp_data[-2]) + 1 
        scn_h = int(tmp_data[-1]) + 1
    # Look for the message marking image onset 
    if 'forcedchoice_start_3' in line:
        trial_start = int(float(tmp_data[1]))
        new_trial = True
        # Store the position and duration of all fixations in lists
        fix_x_testing = []
        fix_y_testing = []
        fix_duration_testing = []
        
        SACC_start_x_testing = []
        SACC_start_y_testing = []
        SACC_duration_testing = []
        SACC_end_x_testing = []
        SACC_end_y_testing = []
        
        BLINK_duration_testing = []
        
    if 'forcedchoice_end_3' in line:
        new_trial = False
        # Add the fixation data for this trial to the all-trials lists
        fix_x_testing_3.extend(fix_x_testing)
        fix_y_testing_3.extend(fix_y_testing)
        fix_duration_testing_3.extend(fix_duration_testing)
        
        SACC_start_x_testing_3.extend(SACC_start_x_testing)
        SACC_start_y_testing_3.extend(SACC_start_y_testing)
        SACC_duration_testing_3.extend(SACC_duration_testing) 
        SACC_end_x_testing_3.extend(SACC_end_x_testing)
        SACC_end_y_testing_3.extend(SACC_end_y_testing)
        
        BLINK_duration_testing_3.extend(BLINK_duration_testing)
    
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'EFIX' in line:
            # EFIX R   349482768	349482935	167	 1312.5	  729.7	   3770
            start_time, end_time, duration_t_f_3, x15, y15 = [int(float(x)) for x in tmp_data[2:7]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_t_f_3 = duration_t_f_3 - (trial_start - start_time)
                    start_time = trial_start
            fix_x_testing.append(x15)
            fix_y_testing.append(y15)
            fix_duration_testing.append(duration_t_f_3) 
            
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        #ESACC R  554166	554204	38	 1207.8	  978.0	 1293.4	  761.3	   8.32	amplitude    517 velocity
        if 'ESACC' in line:
            start_time, end_time, duration_t_s_3, x16, y16,x17,y17 = [int(float(x)) for x in tmp_data[2:9]] 
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_t_s_3 = duration_t_s_3 - (trial_start - start_time)
                    start_time = trial_start
            SACC_start_x_testing_3.append(x16)
            SACC_start_y_testing_3.append(y16)
            SACC_duration_testing_3.append(duration_t_s_3)
            SACC_end_x_testing_3.append(x17)
            SACC_end_y_testing_3.append(y17)
            
    if new_trial:
        # Path to the background image
        # MSG 3558923 !V IMGLOAD FILL images/woods.jpg 
        if 'EBLINK' in line:
            #EBLINK R 565909	565986	77

            start_time, end_time, duration_t_3_b = [int(float(x)) for x in tmp_data[2:]]
            if start_time < trial_start :
                # Adjust the start time and duration of the fixation
                    duration_t_3_b = duration_t_3_b - (trial_start - start_time)
                    start_time = trial_start 
            
            BLINK_duration_testing_3.append(duration_t_3_b)
                      
asc.close()
            # Combine the fixation data lists into a dictionary
eye_data_testing_pic_3 = {
    'fix_x_t_position': fix_x_testing_3,
    'fix_y_t_position': fix_y_testing_3,
    'fix_t_duration': fix_duration_testing_3,
    'SACC_x_t_s_position': SACC_start_x_testing_3,
    'SACC_y_t_s_position': SACC_start_y_testing_3,
    'SACC_t_duration': SACC_duration_testing_3,
    'SACC_x_t_e_position': SACC_end_x_testing_3,
    'SACC_y_t_e_position': SACC_end_y_testing_3,
    'BLINK_t_duration': BLINK_duration_testing_3,
}

# Create a DataFrame from the fixation data dictionary
#df_fixations_0 = pd.DataFrame(fixation_data)

#df = pd.DataFrame.from_dict(d, orient='index')

#print(df)
df_data_testing_pic_3 = pd.DataFrame.from_dict(eye_data_testing_pic_3, orient='index')
print(df_data_testing_pic_3.T)
df_data_testing_pic_3 =df_data_testing_pic_3.T

# Print the first 10 rows of the DataFrame
#df_fixations_3

    fix_x_t_position  fix_y_t_position  fix_t_duration  SACC_x_t_s_position  \
0             2313.0             854.0           149.0               2309.0   
1             1914.0             620.0           139.0               1911.0   
2             1951.0             602.0           258.0               1946.0   
3             1940.0             611.0           322.0               1944.0   
4             1286.0             709.0           256.0               1266.0   
5             1322.0             884.0           170.0               1320.0   
6             1255.0            1323.0           196.0               1252.0   
7             1113.0             886.0           202.0               1117.0   
8              584.0             602.0           206.0                586.0   
9              568.0             581.0          1941.0                583.0   
10             562.0             584.0          1376.0                571.0   
11             595.0             571.0           240

In [1298]:
df_testing = pd.concat([df_data_testing_pic_0, df_data_testing_pic_1, df_data_testing_pic_2, df_data_testing_pic_3]).reset_index()
df_testing

,index,fix_x_t_position,fix_y_t_position,fix_t_duration,SACC_x_t_s_position,SACC_y_t_s_position,SACC_t_duration,SACC_x_t_e_position,SACC_y_t_e_position,BLINK_t_duration
0,0,1259.0,720.0,51.0,1246.0,740.0,6.0,1232.0,737.0,NaN
1,1,1233.0,730.0,175.0,1231.0,724.0,65.0,597.0,593.0,NaN
2,2,593.0,585.0,135.0,589.0,580.0,43.0,1177.0,951.0,NaN
3,3,1152.0,931.0,209.0,1148.0,932.0,18.0,1100.0,926.0,NaN
4,4,1104.0,919.0,309.0,1108.0,904.0,45.0,1009.0,997.0,NaN
...,...,...,...,...,...,...,...,...,...,...
239,15,566.0,203.0,79.0,560.0,198.0,37.0,611.0,583.0,NaN
240,16,595.0,568.0,173.0,591.0,568.0,15.0,579.0,594.0,NaN
241,17,578.0,601.0,130.0,574.0,602.0,44.0,1290.0,998.0,NaN
242,18,1273.0,989.0,142.0,1266.0,990.0,46.0,1264.0,1326.0,NaN


In [1299]:
df_testing = pd.concat([df_data_testing_pic_0, df_data_testing_pic_1, df_data_testing_pic_2, df_data_testing_pic_3]).reset_index()

df_eye_t = {'fix_t_duration':[], 'fix_x_t':[], 'fix_y_t':[],
         'SACC_t_duration':[], 
          'BLINK_t_duration':[], 
          'region_t':[],
         }

for i in range(0,len(df_testing)):
    df_eye_t['fix_t_duration'].append(df_testing['fix_t_duration'][i])
    df_eye_t['fix_x_t'].append(df_testing['fix_x_t_position'][i])
    df_eye_t['fix_y_t'].append(df_testing['fix_y_t_position'][i])
    if 352< df_testing['fix_x_t_position'][i] <928 and 92< df_testing['fix_y_t_position'][i] <412:
        df_eye_t['region_t'].append('left')
    elif 1632< df_testing['fix_x_t_position'][i] <2208 and 92< df_testing['fix_y_t_position'][i] <412:
        df_eye_t['region_t'].append('right')
    elif 992< df_testing['fix_x_t_position'][i] <1568 and 812< df_testing['fix_y_t_position'][i] <1132:
        df_eye_t['region_t'].append('bottom')
    else:
        df_eye_t['region_t'].append('N/A')
            
        
    df_eye_t['SACC_t_duration'].append(df_testing['SACC_t_duration'][i])

    df_eye_t['BLINK_t_duration'].append(df_testing['BLINK_t_duration'][i])
    
        
df_eye_t = pd.DataFrame.from_dict(df_eye_t, orient='index')
print(df_eye_t.T)
df_eye_t = df_eye_t.T
print(df_eye_t)

#df_eye_t.to_csv('df_eye_testing_030705.csv')

    fix_t_duration fix_x_t fix_y_t SACC_t_duration BLINK_t_duration region_t
0             51.0  1259.0   720.0             6.0              NaN      N/A
1            175.0  1233.0   730.0            65.0              NaN      N/A
2            135.0   593.0   585.0            43.0              NaN      N/A
3            209.0  1152.0   931.0            18.0              NaN   bottom
4            309.0  1104.0   919.0            45.0              NaN   bottom
..             ...     ...     ...             ...              ...      ...
239           79.0   566.0   203.0            37.0              NaN     left
240          173.0   595.0   568.0            15.0              NaN      N/A
241          130.0   578.0   601.0            44.0              NaN      N/A
242          142.0  1273.0   989.0            46.0              NaN   bottom
243          203.0  1255.0  1322.0            15.0              NaN      N/A

[244 rows x 6 columns]
    fix_t_duration fix_x_t fix_y_t SACC_t_duration B

In [1300]:
fix_t_left = df_eye_t[df_eye_t['region_t']=='left']['fix_t_duration'].sum()
fix_t_right = df_eye_t[df_eye_t['region_t']=='right']['fix_t_duration'].sum()
fix_t_bottom = df_eye_t[df_eye_t['region_t']=='bottom']['fix_t_duration'].sum()

fix_t_on_pic = df_eye_t[df_eye_t['region_t']!='N/A']['fix_t_duration'].sum()
fix_t_off_pic = df_eye_t[df_eye_t['region_t']=='N/A']['fix_t_duration'].sum()
fix_t_total = df_eye_t['fix_t_duration'].sum()

SACC_t_total = df_eye_t['SACC_t_duration'].sum()

BLINK_t_total = df_eye_t['BLINK_t_duration'].sum()

#eye_t_total = fix_t_total+SACC_t_total+BLINK_t_total
eye_t_total = fix_t_total+SACC_t_total
percent_fix_t_on_pic = fix_t_on_pic/fix_t_total

percent_fix_t_off_pic = fix_t_off_pic/fix_t_total

df_eye_t['percent_fix_t_on_pic'] = percent_fix_t_on_pic
df_eye_t['percent_fix_t_off_pic'] = percent_fix_t_off_pic
df_eye_t['fix_t_total'] = fix_t_total
df_eye_t['SACC_t_total'] = SACC_t_total
df_eye_t['BLINK_t_total'] = BLINK_t_total
df_eye_t['eye_t_total'] = eye_t_total

df_eye_t['fix_t_left'] = fix_t_left
df_eye_t['fix_t_right'] = fix_t_right
df_eye_t['fix_t_bottom'] = fix_t_bottom
df_eye_t['fix_t_on_pic'] = fix_t_on_pic
df_eye_t['fix_t_off_pic'] = fix_t_off_pic

df_eye_testing=df_eye_t
df_eye_testing



,fix_t_duration,fix_x_t,fix_y_t,SACC_t_duration,BLINK_t_duration,region_t,percent_fix_t_on_pic,percent_fix_t_off_pic,fix_t_total,SACC_t_total,BLINK_t_total,eye_t_total,fix_t_left,fix_t_right,fix_t_bottom,fix_t_on_pic,fix_t_off_pic
0,51.0,1259.0,720.0,6.0,NaN,N/A,0.192035,0.807965,95540.0,12374.0,1131.0,107914.0,7222.0,6681.0,4444.0,18347.0,77193.0
1,175.0,1233.0,730.0,65.0,NaN,N/A,0.192035,0.807965,95540.0,12374.0,1131.0,107914.0,7222.0,6681.0,4444.0,18347.0,77193.0
2,135.0,593.0,585.0,43.0,NaN,N/A,0.192035,0.807965,95540.0,12374.0,1131.0,107914.0,7222.0,6681.0,4444.0,18347.0,77193.0
3,209.0,1152.0,931.0,18.0,NaN,bottom,0.192035,0.807965,95540.0,12374.0,1131.0,107914.0,7222.0,6681.0,4444.0,18347.0,77193.0
4,309.0,1104.0,919.0,45.0,NaN,bottom,0.192035,0.807965,95540.0,12374.0,1131.0,107914.0,7222.0,6681.0,4444.0,18347.0,77193.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,79.0,566.0,203.0,37.0,NaN,left,0.192035,0.807965,95540.0,12374.0,1131.0,107914.0,7222.0,6681.0,4444.0,18347.0,77193.0
240,173.0,595.0,568.0,15.0,NaN,N/A,0.192035,0.807965,95540.0,12374.0,1131.0,107914.0,7222.0,6681.0,4444.0,18347.0,77193.0
241,130.0,578.0,601.0,44.0,NaN,N/A,0.192035,0.807965,95540.0,12374.0,1131.0,107914.0,7222.0,6681.0,4444.0,18347.0,77193.0
242,142.0,1273.0,989.0,46.0,NaN,bottom,0.192035,0.807965,95540.0,12374.0,1131.0,107914.0,7222.0,6681.0,4444.0,18347.0,77193.0


In [1301]:
print(fix_t_left, fix_t_right, fix_t_bottom, fix_t_total, SACC_t_total, BLINK_t_total)
print(percent_fix_t_on_pic, percent_fix_t_off_pic, SACC_t_total, BLINK_t_total)

7222.0 6681.0 4444.0 95540.0 12374.0 1131.0
0.1920347498429977 0.8079652501570023 12374.0 1131.0


In [1302]:
df_eye_all = pd.concat([df_eye_encoding, df_eye_testing]).reset_index()
df_eye_all 
df_eye_all.to_csv(r'D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J022306.csv')

In [1305]:
# can paste csv in here for combination


#block_6 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J022306.csv')
block_5 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J022305.csv')
block_4 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J022304.csv')
block_3 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J022303.csv')
block_2 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J022302.csv')
block_1 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J022301.csv')

df_all = pd.concat([block_1,block_2,block_3,block_4,block_5]).reset_index()
df_all.to_csv(r'D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0223.csv')
df_all

,level_0,Unnamed: 0,index,fix_e_duration,fix_x_e,fix_y_e,SACC_e_duration,BLINK_e_duration,region_e,percent_fix_e_on_pic,...,percent_fix_t_off_pic,fix_t_total,SACC_t_total,BLINK_t_total,eye_t_total,fix_t_left,fix_t_right,fix_t_bottom,fix_t_on_pic,fix_t_off_pic
0,0,0,0,38.0,302.0,485.0,49.0,5.0,off_pic_e,0.482936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1,52.0,300.0,479.0,9.0,NaN,off_pic_e,0.482936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,2,161.0,1247.0,685.0,60.0,NaN,on_pic_e,0.482936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,3,242.0,1352.0,680.0,28.0,NaN,on_pic_e,0.482936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,4,513.0,1130.0,671.0,30.0,NaN,on_pic_e,0.482936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1584,342,342,163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.742619,55754.0,7526.0,308.0,63280.0,2338.0,2615.0,9397.0,14350.0,41404.0
1585,343,343,164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.742619,55754.0,7526.0,308.0,63280.0,2338.0,2615.0,9397.0,14350.0,41404.0
1586,344,344,165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.742619,55754.0,7526.0,308.0,63280.0,2338.0,2615.0,9397.0,14350.0,41404.0
1587,345,345,166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.742619,55754.0,7526.0,308.0,63280.0,2338.0,2615.0,9397.0,14350.0,41404.0


In [1306]:
# # can paste csv in here for combination
block_13 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0223.csv').drop(columns=['level_0'])
block_12 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0224.csv').drop(columns=['level_0'])
block_11 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0225.csv').drop(columns=['level_0'])
block_10 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0226.csv').drop(columns=['level_0'])
block_9 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0227.csv').drop(columns=['level_0'])
block_8 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0228.csv').drop(columns=['level_0'])
block_7 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0301.csv').drop(columns=['level_0'])
block_6 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0302.csv').drop(columns=['level_0'])
block_5 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0303.csv').drop(columns=['level_0'])
block_4 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0304.csv').drop(columns=['level_0'])
block_3 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0305.csv').drop(columns=['level_0'])
block_2 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0306.csv').drop(columns=['level_0'])
block_1 = pd.read_csv('D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J0307.csv').drop(columns=['level_0'])

df_all_1 = pd.concat([block_1,block_2,block_3,block_4,block_5,block_6,block_7,block_8,block_9,block_10,block_11,block_12,block_13]).reset_index()
df_all_1.to_csv(r'D:/Jupiter_eyemovement_data/Pic_TOJ/formal/SESSION1/CSV/J_SESSION1_ALL.csv')
df_all_1


,level_0,Unnamed: 0,Unnamed: 0.1,index,fix_e_duration,fix_x_e,fix_y_e,SACC_e_duration,BLINK_e_duration,region_e,...,percent_fix_t_off_pic,fix_t_total,SACC_t_total,BLINK_t_total,eye_t_total,fix_t_left,fix_t_right,fix_t_bottom,fix_t_on_pic,fix_t_off_pic
0,0,0,0,0,181.0,1266.0,715.0,28.0,28.0,on_pic_e,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1,1,199.0,1230.0,593.0,29.0,130.0,on_pic_e,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,2,2,240.0,1001.0,612.0,19.0,NaN,on_pic_e,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,3,3,233.0,1058.0,661.0,29.0,NaN,on_pic_e,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,4,4,250.0,1217.0,617.0,28.0,NaN,on_pic_e,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20795,1584,1584,342,163,NaN,NaN,NaN,NaN,NaN,NaN,...,0.742619,55754.0,7526.0,308.0,63280.0,2338.0,2615.0,9397.0,14350.0,41404.0
20796,1585,1585,343,164,NaN,NaN,NaN,NaN,NaN,NaN,...,0.742619,55754.0,7526.0,308.0,63280.0,2338.0,2615.0,9397.0,14350.0,41404.0
20797,1586,1586,344,165,NaN,NaN,NaN,NaN,NaN,NaN,...,0.742619,55754.0,7526.0,308.0,63280.0,2338.0,2615.0,9397.0,14350.0,41404.0
20798,1587,1587,345,166,NaN,NaN,NaN,NaN,NaN,NaN,...,0.742619,55754.0,7526.0,308.0,63280.0,2338.0,2615.0,9397.0,14350.0,41404.0
